# 🌾 AI-Powered Crop Yield Prediction
This notebook trains two models:
- Random Forest (Scikit-learn)
- DNN (TensorFlow)

It uses synthetic agricultural sensor data to predict crop yield, and converts the DNN to TensorFlow Lite for deployment on edge devices.

In [ ]:
# Step 1: Generate mock dataset
import pandas as pd
import numpy as np

np.random.seed(42)
n = 500
data = {
    "temperature": np.random.uniform(20, 35, n),
    "humidity": np.random.uniform(40, 80, n),
    "soil_moisture": np.random.uniform(0.2, 0.6, n),
    "light": np.random.uniform(800, 1600, n),
    "ndvi": np.random.uniform(0.4, 0.9, n),
    "days_since_planting": np.random.randint(30, 120, n),
}
data["yield"] = (
    0.03 * data["temperature"] +
    0.02 * data["humidity"] +
    0.1 * np.array(data["soil_moisture"]) * 100 +
    0.001 * data["light"] +
    5 * data["ndvi"] +
    0.01 * data["days_since_planting"] +
    np.random.normal(0, 1, n)
)

df = pd.DataFrame(data)
df.to_csv("crop_data.csv", index=False)
df.head()

In [ ]:
# Step 2: Random Forest Regression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv("crop_data.csv")
X = df.drop(columns=["yield"])
y = df["yield"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
print("Random Forest R²:", r2_score(y_test, y_pred))
print("Random Forest RMSE:", mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
# Step 3: DNN Model with TensorFlow
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

dnn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

dnn_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
history = dnn_model.fit(X_train_scaled, y_train, epochs=50, validation_split=0.2, verbose=1)
loss, mae = dnn_model.evaluate(X_test_scaled, y_test)
print("Test MAE:", mae)

In [ ]:
# Step 4: Plot training loss
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Step 5: Convert to TensorFlow Lite
dnn_model.save("crop_yield_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(dnn_model)
tflite_model = converter.convert()

with open("crop_yield_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved.")